In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Base de dados

In [2]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"base_dados\Cel Manfrini.csv"

caminho_2 = r"base_dados\Cel Manfrini(1).csv"

caminho_3 = r"base_dados\Cel Manfrini(2).csv"

caminho_4 = r"base_dados\Cel Manfrini(3).csv"

# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Pesquisas no DataFrame

In [3]:
# Criando um DataFrame para testes 
df_teste = df.copy()

df_teste.head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,01590728000850,MICROTECNICA INFORMATICA LTDA,2022-11-03,335,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2022NE001245,I3FPPREDEGE,DESPESAS GERAIS DAS UGE - GESTÃO EME.,0032,171460,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3FP,PRE,DEGE,0.0,0.0,0.0,14637.78
1,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,01590728000850,MICROTECNICA INFORMATICA LTDA,2022-12-05,303,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2022NE002430,IXAPFUNDGOA,ADMINISTRACAO DAS OM,0032,171502,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160507,EME-GESTOR,IXAP,FUN,DGOA,0.0,0.0,0.0,21956.67


## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [4]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

df_nd30_100dias.to_excel('nd30_PA.xlsx')

### Empenhos ND 52

In [5]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

df_nd52_100dias.to_excel('nd52_PA.xlsx')

# Estudo DIEx Nº 904-SGS/SDIR/DIR 

### Previsão de Recursos Orçamentários (PRO) 2024 - Apoio Administrativo

## Quanto aos contratos Administrativos

#### Recursos FUNADOM

In [6]:
filtro = ((df_teste['PI'] == 'I3DAFUNADOM'))

df_funadom = df_teste.loc[filtro].sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)
df_funadom.head()

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,09249668000141,INDUSTRIA GRAFICA ESCALA LTDA,2023-09-25,9,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2023NE001193,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,160073,DGO,I3DA,FUN,ADOM,20606.08,0.0,20606.08,0.0
1,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,65271868000171,OXI QUIMICA LTDA,2023-09-13,21,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001150,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,8996.00,0.0,8996.00,0.0
2,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,23178866000192,OPPORTUNITAT COMERCIO E SERVICOS LTDA,2023-08-21,44,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903920,MANUT.E CONS.DE B.MOVEIS DE OUTRAS NATUREZAS,2023NE001035,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,8675.00,0.0,8675.00,0.0
3,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,49577909000104,COMERCIAL RB - COMERCIO VAREJISTA E ATACADISTA...,2023-09-13,21,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001152,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160073,DGO,I3DA,FUN,ADOM,6600.00,0.0,6600.00,0.0
4,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,27619368000199,TOTUS PISOS E AZULEJOS LTDA,2023-09-18,16,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE001184,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160073,DGO,I3DA,FUN,ADOM,5094.70,0.0,5094.70,0.0


In [7]:
# Valores Empenhados no Exércicio Corrente para FUNADOM
total_funadom = df_funadom['TOTAL_A_PAGAR'].sum() + df_funadom['PAGO'].sum()

print(f'Total de Recursos empenhados para FUNADOM: {locale.currency(total_funadom, grouping=True)}')

Total de Recursos empenhados para FUNADOM: R$ 392.666,27


#### Contratos de Pequenos Vultos

In [8]:
# Filtrar os PI específicos para os contratos

filtro = ((df_teste['PI'] == 'I3DAFUNCOPI') | 
         (df_teste['PI'] == 'I3DAFUNLARO') | 
         (df_teste['PI'] == 'I3DAFUNPETA') | 
         (df_teste['PI'] == 'I3DAFUNELEV') | 
         (df_teste['PI'] == 'I3DAFUNDEDE'))

df_teste.loc[filtro]

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


#### Contratos de Grande Vulto

- Não está autorizado celebrar novos contratos

In [9]:
filtro = ((df_teste['PI'] == 'I3DACNTLICO') | 
         (df_teste['PI'] == 'I3DACNTMABM') | 
         (df_teste['PI'] == 'I3DACNTMABI'))

df_grande_vulto = df_teste.loc[filtro].sort_values(by='A_LIQUIDAR', ascending=False).reset_index()
df_grande_vulto.head()

,index,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,158,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,20584944000160,TBL SERVICOS LTDA,2023-06-14,112,160533,4º CGCFEX,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000661,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,CNT,LICO,72837.98,0.0,72837.98,0.0
1,157,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,20584944000160,TBL SERVICOS LTDA,2023-05-26,131,160533,4º CGCFEX,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000552,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,0032,171460,ANO,GLOBAL,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",MG,MG,160073,DGO,I3DA,CNT,LICO,68931.48,0.0,68931.48,0.0
2,161,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,20584944000160,TBL SERVICOS LTDA,2023-08-15,50,160533,4º CGCFEX,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000994,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,CNT,LICO,36418.99,0.0,36418.99,0.0
3,160,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,20584944000160,TBL SERVICOS LTDA,2023-07-05,91,160533,4º CGCFEX,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000849,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,CNT,LICO,36418.99,0.0,36418.99,0.0
4,156,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,20584944000160,TBL SERVICOS LTDA,2023-04-14,173,160533,4º CGCFEX,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000374,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,CNT,LICO,34465.74,0.0,34465.74,172328.7


In [10]:
# Total de empenhos para  Contrato de Grandes Vulto
df_grande_vulto.shape[0]
print(f'Total de empenhos firmados ao longo de 2023: {df_grande_vulto.shape[0]}')

Total de empenhos firmados ao longo de 2023: 21


In [11]:
# Valores Empenhados no Exércicio Corrente para Contratos de Grande Vulto
total_grande_vulto = df_grande_vulto['TOTAL_A_PAGAR'].sum() + df_grande_vulto['PAGO'].sum()

print(f'Total de Recursos Alocados para Contratos de Grande Vulto: {locale.currency(total_grande_vulto, grouping=True)}')

Total de Recursos Alocados para Contratos de Grande Vulto: R$ 1.062.245,76


In [12]:
df_recursos = pd.read_excel(r'C:\Users\User\OneDrive\Trabalho\ESA 2023\Dados\Dash_SAG_Novo\base_dados\Recursos_Disponiveis.xlsx', sheet_name='Recursos_ESA')
df_recursos = df_recursos[['ND', 'UG Emissora', 'PI', 'Descrição PI', 'Detalhe Descrição PI', 'Soma de Valor']]

# Convertendo tipos de colunas
df_recursos['ND'] =  df_recursos['ND'].astype(str)
df_recursos['UG Emissora'] = df_recursos['UG Emissora'].astype(str)
df_recursos['UG Emissora'] = df_recursos['UG Emissora'].apply(lambda x: x.replace('.0', ''))

c:\Users\User\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\User\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


In [13]:
locale.currency(df_recursos['Soma de Valor'].sum(), grouping=True)

'R$ 430.112,08'

In [14]:
# Recursos disponíveis por ND
df_aux = df_recursos.loc[:,['ND', 'PI','Soma de Valor']].groupby(by= ['ND']).sum().sort_values(by='Soma de Valor', ascending=False).reset_index()

fig = px.bar(df_aux,
             x = 'ND',
             y = 'Soma de Valor',
             title='Recurso a ser empenhado por ND',
             labels={'ND':'Natureza da Despesa', 'Soma de Valor':'Recurso para empenhar'},
             text_auto='.3s',
             height=500,
             color='Soma de Valor',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [15]:
# Recursos disponíveis por PI
df_aux = df_recursos.loc[:,['ND', 'PI','Soma de Valor']].groupby(by= ['PI']).sum().sort_values(by='Soma de Valor', ascending=False).reset_index()

fig = px.bar(df_aux.head(10),
             x = 'PI',
             y = 'Soma de Valor',
             title='10 maiores Recursos para empenho',
             labels={'PI':'Programa Interno', 'Soma de Valor':'Recurso para empenhar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='Soma de Valor',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [16]:
df_aux = df_recursos.loc[:,['ND', 'PI','Soma de Valor']].groupby(by= ['ND']).sum().sort_values(by='Soma de Valor', ascending=False).reset_index()

fig = px.pie(df_aux, values='Soma de Valor', 
                     names='ND', 
                     title='Porcentagem de Recurso por ND',
                     color_discrete_sequence=px.colors.sequential.RdBu,
                     template='plotly_dark')
fig.update_traces(textposition='outside',
                  hole=.6,
                  textinfo='percent+label',
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update(layout_showlegend=False)

fig.show()

In [17]:
fig = px.colors.sequential.swatches()
fig.show()

#### Consulta a NE

In [18]:
natureza = ['339147', '339036', '339031', '339033', '339039']

df_estudo = df_teste[df_teste['ND'].isin(natureza)]

df_estudo = df_estudo[df_estudo['A_LIQUIDAR'] != 0].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_estudo.head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,15072268000106,SGA TUDO PARA SEU EVENTO LTDA,2022-07-05,456,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903914,LOCACAO BENS MOV. OUT.NATUREZAS E INTANGIVEIS,2022NE000742,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,180.00,0.0,180.00,16640.00
1,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,2022-10-06,363,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE001085,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,11086.68,0.0,11086.68,22173.32


In [19]:
# Filtro para empenhos ordinários
df_estudo[df_estudo['TIPO_NE'] == 'ORDINARIO']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
2,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,26669742000106,J. T. CATALDO CASTRO CRIATIVA DIGITAL,2022-11-30,308,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2022NE001843,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RJ,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,1016.25,0.0,1016.25,0.0
3,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,26669742000106,J. T. CATALDO CASTRO CRIATIVA DIGITAL,2022-11-30,308,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2022NE002421,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RJ,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,5758.75,0.0,5758.75,0.0
20,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,04087710000151,MWM ART LTDA,2023-03-08,210,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2023NE000206,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,680.00,0.0,680.00,0.0
21,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,13759849000195,INDUSTRIA FENIX CORTE A LASER LTDA,2023-03-08,210,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2023NE000208,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,3280.00,0.0,3280.00,0.0
31,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,08593332000139,EQUALLI SERVICOS VETERINARIOS LTDA,2023-04-14,173,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE000377,E6RVPLJMTOE,MEDIDAS PROFILÁT SAÚDE EQUINOS,6012,193910,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160504,COEX/GESTOR,E6RV,PLJ,MTOE,480.00,0.0,480.00,0.0
35,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,2023-05-05,152,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2023NE000491,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,480.00,0.0,480.00,0.0
37,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,92221589000176,IRMAOS KOPERECK LTDA,2023-05-18,139,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903309,TRANSPORTE DE SERVIDORES,2023NE000526,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RS,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,86451.40,0.0,86451.40,0.0
51,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,11532912000120,RONALDO RIBEIRO VIEIRA 79989918600,2023-06-29,97,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903920,MANUT.E CONS.DE B.MOVEIS DE OUTRAS NATUREZAS,2023NE000787,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,1091.00,0.0,1091.00,0.0
60,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,28683271000108,C.H.QUEIROZ PECAS E ACESSORIOS LTDA,2023-07-14,82,160533,4º CGCFEX,339031,"PREMIACOES CULT., ART., CIENT., DESP. E OUTR.",33903101,PREMIACOES CULTURAIS,2023NE000877,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,PR,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,2992

In [20]:
notas_empenho = ['2022NE000742', '2022NE001085', '2023NE000002', '2022NE001843', '2023NE000208', '2022NE002702']

df_teste[df_teste['NE'].isin(notas_empenho)]

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
748,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,14807420000199,FACTO TURISMO LTDA,2023-01-11,266,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903302,PASSAGENS PARA O EXTERIOR,2023NE000002,DGPREQTRNP3,MIS EXT REQS TRNP BAG INTER,6012,171389,ANO,GLOBAL,ESA,SP,MG,160505,DGP,DGPR,EQT,RNP3,0.00,0.00,0.00,5047.00
1075,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,15072268000106,SGA TUDO PARA SEU EVENTO LTDA,2022-07-05,456,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903914,LOCACAO BENS MOV. OUT.NATUREZAS E INTANGIVEIS,2022NE000742,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,180.00,0.00,180.00,16640.00
1076,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-28,280,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002702,C1ENMILOUTR,OUTROS,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160503,DECEX/GESTOR,C1EN,MIL,OUTR,12733.76,0.00,12733.76,0.00
1093,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,2022-10-06,363,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE001085,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,11086.68,0.00,11086.68,22173.32
1153,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,13759849000195,INDUSTRIA FENIX CORTE A LASER LTDA,2023-03-08,210,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2023NE000208,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,3280.00,0.00,3280.00,0.00
1317,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,26669742000106,J. T. CATALDO CASTRO CRIATIVA DIGITAL,2022-11-30,308,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2022NE001843,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RJ,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,1016.25,0.00,1016.25,0.00
1354,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,167129,NaN,2023-01-30,247,160533,4º CGCFEX,339015,DIARIAS - PESSOAL MILITAR,33901514,DIARIAS NO PAIS,2023NE000208,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,0,NaN,MG,167129,ESA,C1EN,CON,DETM,32685.03,17685.96,50370.99,17629.01
2886,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,50429810000136,SAPRA LANDAUER SERVICO DE ASSESSORIA E PROTECA...,2023-01-11,266,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE000002,D8SACNTDOSI,SERVICOS DE PROTECAO A RADIACAO IONIZANTE,0032,215842,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,167505,DGP,D8SA,CNT,DOSI,447.72,0.00,447.72,895.44


In [23]:
df_acao = df_teste[df_teste['ACAO'] == '8965']

In [24]:
df_acao = df_acao[df_acao['A_LIQUIDAR'] != 0]

In [26]:
df_acao['A_LIQUIDAR'].sum()

1226616.0899999999

In [27]:
locale.currency(df_acao['A_LIQUIDAR'].sum(), grouping=True)

'R$ 1.226.616,09'

In [31]:
col = ['ND', 'A_LIQUIDAR']
df_acao.loc[:,col].groupby(by='ND').sum().reset_index().sort_values(by='A_LIQUIDAR', ascending=False)

,ND,A_LIQUIDAR
3,339033,324553.45
4,339039,301473.80
6,339093,245770.00
5,339040,214747.64
7,449052,68905.10
1,339030,62216.44
0,339015,5957.06
2,339031,2992.60


In [33]:
df_acao[df_acao['ND'] == '339093']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
1179,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,160129,NaN,2023-05-16,141,160533,4º CGCFEX,339093,INDENIZACOES E RESTITUICOES,33909314,RESSARCIMENTO DE PASSAGENS E DESP.C/LOCOMOCAO,2023NE000508,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,0,NaN,MG,160129,ESA,C4EN,MIL,CAPE,176770.0,0.0,176770.0,0.0
1180,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,160129,NaN,2023-06-28,98,160533,4º CGCFEX,339093,INDENIZACOES E RESTITUICOES,33909302,RESTITUICOES,2023NE000785,C1ENMILOUTR,OUTROS,6012,171423,ANO,ORDINARIO,0,NaN,MG,160129,ESA,C1EN,MIL,OUTR,69000.0,0.0,69000.0,0.0
